In [56]:
import json
import xmltodict
import pandas as pd
import numpy as np
import unidecode

In [57]:
# Função para retirar acentuação 
def f(str):
    return (unidecode.unidecode(str))

Formato do float

In [29]:
pd.options.display.float_format = '{:,.2f}'.format

## Estudos sobre a verba de gabinete dos deputados estaduais da atual legislatura

Fonte
https://www.al.sp.gov.br/dados-abertos/recurso/21

In [30]:
with open("dados/deputados/despesas_gabinetes_julho_2018.xml", 'r') as f:
    xmlString = f.read()
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
with open("output.json", 'w') as f:
    f.write(jsonString)

In [31]:
arquivo_json = open('output.json', 'r')

In [32]:
dados_json = json.load(arquivo_json)

In [33]:
lista = dados_json['despesas']['despesa']

In [34]:
df_despesas = pd.DataFrame(lista, columns = ['Ano', 'CNPJ', 'Deputado', 'Fornecedor', 'Matricula', 'Mes', 'Tipo', 'Valor'])

In [35]:
df_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455156 entries, 0 to 455155
Data columns (total 8 columns):
Ano           455156 non-null object
CNPJ          445475 non-null object
Deputado      455156 non-null object
Fornecedor    455156 non-null object
Matricula     455156 non-null object
Mes           455156 non-null object
Tipo          455156 non-null object
Valor         455156 non-null object
dtypes: object(8)
memory usage: 27.8+ MB


Rotinas de limpeza

In [36]:
df_despesas['Valor'] = df_despesas.Valor.astype('float64')

In [37]:
df_despesas['CNPJ'] = df_despesas.CNPJ.astype('str')

In [38]:
def limpeza(linha):
    documento = linha["CNPJ"]
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf_"

In [39]:
df_despesas['cnpj_raiz_fornecedor'] = df_despesas.apply(limpeza, axis = 1)

In [40]:
df_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455156 entries, 0 to 455155
Data columns (total 9 columns):
Ano                     455156 non-null object
CNPJ                    455156 non-null object
Deputado                455156 non-null object
Fornecedor              455156 non-null object
Matricula               455156 non-null object
Mes                     455156 non-null object
Tipo                    455156 non-null object
Valor                   455156 non-null float64
cnpj_raiz_fornecedor    455156 non-null object
dtypes: float64(1), object(8)
memory usage: 31.3+ MB


In [41]:
df_despesas.reset_index().head()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,0,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,200.00,71806251
1,1,2015,68064740000125,ABELARDO CAMARINHA,PAPER FACE COMERCIAL LTDA,300257,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,295.40,68064740
2,2,2015,22145388877,ABELARDO CAMARINHA,LARA SERVINO VARGAS ABREU,300257,3,N - MORADIA,"2,850.00",documento_cpf_
3,3,2015,10915883000113,ABELARDO CAMARINHA,CENTRO AUTOMOTIVO TOP 23 LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,100.00,10915883
4,4,2015,71806251000106,ABELARDO CAMARINHA,AUTO POSTO DE SERVIÇOS MARV LTDA,300257,4,A - COMBUSTÍVEIS E LUBRIFICANTES,195.01,71806251


Soma histórico de despesas

In [44]:
df_despesas['Valor'].sum()

311900200.82000023

In [45]:
"${:,.2f}".format(df_despesas.Valor.sum())

'$311,900,200.82'

Soma por deputados 

In [46]:
df_despesas.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Deputado,Valor
41,CAMPOS MACHADO,"4,066,365.58"
10,ALDO DEMARCHI,"3,801,287.66"
183,MARIA LÚCIA AMARY,"3,762,815.87"
5,AFONSO LOBATO,"3,727,427.14"
72,CÉLIA LEÃO,"3,675,538.21"
234,ROBERTO ENGLER,"3,668,365.75"
160,LUIZ CARLOS GONDIM,"3,668,202.46"
100,ENIO TATTO,"3,621,293.14"
20,ANALICE FERNANDES,"3,593,955.27"
17,ANA DO CARMO,"3,422,842.43"


Soma por fornecedor

In [18]:
df_despesas.groupby("Fornecedor").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Fornecedor,Valor
66100,TELEFONICA,"2,911,294.99"
66107,TELEFONICA BRASIL S.A.,"2,500,701.86"
47288,NEXTEL TELECOMUNICAÇÕES LTDA,"2,179,388.71"
66120,TELEFÔNICA BRASIL S.A.,"2,160,877.74"
66105,TELEFONICA BRASIL S.A,"1,597,810.51"
50926,PEDÁGIO,"1,322,905.88"
68997,VIVO S.A.,"1,255,101.17"
47289,NEXTEL TELECOMUNICAÇÕES LTDA.,"1,252,512.74"
66108,TELEFONICA BRASIL S/A,"1,219,905.04"
68988,VIVO,"1,089,259.35"


Tabela dinâmica por ano, deputado e cnpj raiz de fornecedor

In [19]:
#pd.pivot_table(df_despesas,index=["Ano","Deputado", "cnpj_raiz_fornecedor"], values=["Valor"], aggfunc=np.sum)

In [20]:
df_despesas.to_csv('resultados/despesas_dep_est_sp_julho.csv', index=False, decimal=',')

# Carrega arquivo salvo

In [25]:
df_despesas = pd.read_csv("resultados/despesas_dep_est_sp_julho.csv", sep=',',encoding = 'utf-8', converters={'CNPJ': lambda x: str(x), 'cnpj_raiz_fornecedor': lambda x: str(x), 'Ano': lambda x: str(x)}, decimal=',')

In [26]:
df_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455156 entries, 0 to 455155
Data columns (total 9 columns):
Ano                     455156 non-null object
CNPJ                    455156 non-null object
Deputado                455156 non-null object
Fornecedor              455156 non-null object
Matricula               455156 non-null int64
Mes                     455156 non-null int64
Tipo                    455156 non-null object
Valor                   455156 non-null float64
cnpj_raiz_fornecedor    455156 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 31.3+ MB


Procura por nome de fornecedor

In [20]:
mask = df_despesas['Fornecedor'].str.contains("ITORORO", na=False)
procura = df_despesas[mask]

In [21]:
procura.reset_index().head()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,41,2015,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,9,A - COMBUSTÍVEIS E LUBRIFICANTES,56.28,61488904
1,42,2015,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,9,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,"3,243.72",61488904
2,65,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,3,A - COMBUSTÍVEIS E LUBRIFICANTES,210.00,61488904
3,66,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,3,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,"3,148.71",61488904
4,105,2016,61488904000364,ABELARDO CAMARINHA,ITORORO VEÍCULOS E PEÇAS LTDA,300257,8,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,"2,454.56",61488904


Procura por CNPJ raiz

In [22]:
nome = df_despesas[(df_despesas['cnpj_raiz_fornecedor'] == '19876022')]

In [23]:
nome.reset_index().head()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
0,2636,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,10,K - LOCAÇÃO DE BENS MÓVEIS,"3,800.00",19876022
1,2639,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,10,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...","3,500.00",19876022
2,2647,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,11,K - LOCAÇÃO DE BENS MÓVEIS,"3,800.00",19876022
3,2650,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,11,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...","3,500.00",19876022
4,2659,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,12,K - LOCAÇÃO DE BENS MÓVEIS,"3,800.00",19876022


CNPJ neste caso é um CPF de deputado

In [47]:
nome = df_despesas[(df_despesas['CNPJ'] == '02674980800')]
nome

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
372417,2017,02674980800,ROBERTO MASSAFERA,CIRO BERBES JUNIOR,300478,8,N - MORADIA,"2,500.00",documento_cpf_


Separa lote para checar em https://www.al.sp.gov.br/deputados/prestacao-de-contas/

In [26]:
despesa_selecionada = df_despesas[(df_despesas['Deputado'] == 'ÍTALO CARDOSO') & 
                              (df_despesas['Ano'] == 2007) & 
                              (df_despesas['Mes'] == 3) & 
                              (df_despesas['Tipo'] == 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO')]

In [27]:
despesa_selecionada.Valor.sum()

2632.4100000000003

In [28]:
"${:,.0f}".format(despesa_selecionada.Valor.sum())

'$2,632'

In [29]:
despesa_selecionada

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
455142,2007,00000998000124,ÍTALO CARDOSO,HIPE COMERCIO DE PAPEIS LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,"2,022.30",00000998
455143,2007,00986363000148,ÍTALO CARDOSO,COME'X COMERCIO E REPRESENTAÇÃO LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,500.00,00986363
455144,2007,03476811002103,ÍTALO CARDOSO,DIA BRASIL SOCIEDADE LTDA,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,14.85,03476811
455145,2007,43283811003841,ÍTALO CARDOSO,KALUNGA COM. E IND.GRÁFICA LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,24.30,43283811
455146,2007,47508411090988,ÍTALO CARDOSO,CIA. BRASILEIRA DE DISTRIBUIÇÃO,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,46.19,47508411
455147,2007,61072898000106,ÍTALO CARDOSO,EMPRESA DE AGUAS PETROPOLIS PAULISTA LTDA.,300441,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,24.77,61072898


# Seleciona despesas da atual legislatura

In [48]:
df_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455156 entries, 0 to 455155
Data columns (total 9 columns):
Ano                     455156 non-null object
CNPJ                    455156 non-null object
Deputado                455156 non-null object
Fornecedor              455156 non-null object
Matricula               455156 non-null object
Mes                     455156 non-null object
Tipo                    455156 non-null object
Valor                   455156 non-null float64
cnpj_raiz_fornecedor    455156 non-null object
dtypes: float64(1), object(8)
memory usage: 31.3+ MB


In [49]:
despesas_atuais = df_despesas[(df_despesas['Ano'] == '2015') | 
                              (df_despesas['Ano'] == '2016') | 
                              (df_despesas['Ano'] == '2017') | 
                              (df_despesas['Ano'] == '2018')]

In [50]:
#despesas_atuais["Deputado"] = despesas_atuais["Deputado"].apply(f)

In [51]:
despesas_atuais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82823 entries, 0 to 452412
Data columns (total 9 columns):
Ano                     82823 non-null object
CNPJ                    82823 non-null object
Deputado                82823 non-null object
Fornecedor              82823 non-null object
Matricula               82823 non-null object
Mes                     82823 non-null object
Tipo                    82823 non-null object
Valor                   82823 non-null float64
cnpj_raiz_fornecedor    82823 non-null object
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


In [42]:
despesas_atuais.to_excel('resultados/despesas_atuais.xlsx',sheet_name='Sheet1')

Exemplo de deputado da legislatura passada que registrou algumas despesas em 2015

In [31]:
nome = despesas_atuais[(despesas_atuais['Deputado'] == 'BRUNO COVAS')]
nome

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor
66361,2015,02928132000103,BRUNO COVAS,CENTRO DE SERVIÇOS FRANGO ASSADO SULESTE LTDA,300481,1,A - COMBUSTÍVEIS E LUBRIFICANTES,123.13,02928132
66362,2015,10318092000106,BRUNO COVAS,AUTO POSTO DUQUE SUECIA LTDA.,300481,1,A - COMBUSTÍVEIS E LUBRIFICANTES,115.07,10318092
66363,2015,10559732000170,BRUNO COVAS,AUTO POSTO NAUTICA FREI GASPAR LTDA-ME,300481,1,A - COMBUSTÍVEIS E LUBRIFICANTES,128.17,10559732
66364,2015,43283811001474,BRUNO COVAS,KALUNGA COMERCIO INDUSTRIA GRAFICA LTDA,300481,1,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,48.20,43283811
66365,2015,40432544000147,BRUNO COVAS,CLARO CLUBE,300481,1,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...",735.32,40432544
66366,2015,02575829000148,BRUNO COVAS,AVIANCA,300481,1,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...","1,660.00",02575829
66367,2015,nan,BRUNO COVAS,TAXI,300481,1,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",220.00,documento_cpf_
66368,2015,05742944000158,BRUNO COVAS,MIRAGE EMPREENDIMENTOS E PARTICIPAÇÕES LTDA,300481,1,N - MORADIA,"2,850.00",05742944


## Separa nomes de deputados que têm gastos na atual legislatura para criar um dataframe base de nomes que será finalizado em alesp_unificacodigos

In [47]:
nomes = despesas_atuais['Deputado'].value_counts()

nomes_deputados_atual_legislatura = pd.DataFrame({'Deputado':nomes.index, 'list':nomes.values})

nomes_deputados_atual_legislatura = nomes_deputados_atual_legislatura.drop('list', axis=1)

nomes_deputados_atual_legislatura.info()

nomes_deputados_atual_legislatura.to_csv("resultados/nomes_deputados_atual_legislatura.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 1 columns):
Deputado    136 non-null object
dtypes: object(1)
memory usage: 1.1+ KB


### Carrega nomes de deputados da atual legislatura - gerados no alesp_unificacodigos

In [58]:
nomes_deputados_atual_legislatura = pd.read_csv('resultados/deputados_selecionados_final.csv', converters={'sequencial': lambda x: str(x), 'cpf': lambda x: str(x), 'codigo_unico': lambda x: str(x) })

In [59]:
nomes_deputados_atual_legislatura["nome_deputado"] = nomes_deputados_atual_legislatura["nome_deputado"].apply(f)

In [60]:
nomes_deputados_atual_legislatura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
codigo_unico       120 non-null object
nome_deputado      120 non-null object
uf                 120 non-null object
nome_completo      120 non-null object
sequencial         120 non-null object
cpf                120 non-null object
nome_urna          120 non-null object
partido_eleicao    120 non-null object
situacao           120 non-null object
dtypes: object(9)
memory usage: 8.5+ KB


Filtra por esses nomes

In [61]:
despesas_atuais_final = pd.merge(despesas_atuais, nomes_deputados_atual_legislatura, left_on='Deputado', right_on='nome_deputado')

In [62]:
despesas_atuais_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62566 entries, 0 to 62565
Data columns (total 18 columns):
Ano                     62566 non-null object
CNPJ                    62566 non-null object
Deputado                62566 non-null object
Fornecedor              62566 non-null object
Matricula               62566 non-null object
Mes                     62566 non-null object
Tipo                    62566 non-null object
Valor                   62566 non-null float64
cnpj_raiz_fornecedor    62566 non-null object
codigo_unico            62566 non-null object
nome_deputado           62566 non-null object
uf                      62566 non-null object
nome_completo           62566 non-null object
sequencial              62566 non-null object
cpf                     62566 non-null object
nome_urna               62566 non-null object
partido_eleicao         62566 non-null object
situacao                62566 non-null object
dtypes: float64(1), object(17)
memory usage: 9.1+ MB


Checa alguns nomes

In [67]:
despesa_selecionada = despesas_atuais_final[(despesas_atuais_final['Deputado'] == 'ADILSON ROSSI') & 
                              (despesas_atuais_final['Ano'] == '2015') & 
                              (despesas_atuais_final['Mes'] == '10') & 
                              (despesas_atuais_final['Tipo'] == 'K - LOCAÇÃO DE BENS MÓVEIS')]

In [68]:
despesa_selecionada

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
372,2015,07288003000111,ADILSON ROSSI,PHT COMERCIO E SERVIÇOS DE MAQUINAS PARA CAFE ...,300432,10,K - LOCAÇÃO DE BENS MÓVEIS,620.00,07288003,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA
373,2015,19876022000148,ADILSON ROSSI,CLEAN WORK EQPT E SERV EIRELI ME,300432,10,K - LOCAÇÃO DE BENS MÓVEIS,"3,800.00",19876022,440,ADILSON ROSSI,SP,ADILSON ROSSI,250000001161,01660477832,ADILSON ROSSI,PSB,ELEITO POR MÉDIA


In [73]:
despesa_selecionada = despesas_atuais_final[(despesas_atuais_final['Deputado'] == 'GIL LANCASTER') & 
                              (despesas_atuais_final['Ano'] == '2018') & 
                              (despesas_atuais_final['Mes'] == '3') & 
                              (despesas_atuais_final['Tipo'] == 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO')]
despesa_selecionada

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
26268,2018,07686203000122,GIL LANCASTER,CRIDON COMERCIAL LTDA EPP,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,32.00,07686203,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26269,2018,15359521000107,GIL LANCASTER,O MUNDO DAS EMBALAGENS LTDA - EPP,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,147.20,15359521,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26270,2018,24422336000100,GIL LANCASTER,EDINAURA BATISTA FIGUEIREDO SILVA,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,80.16,24422336,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26271,2018,33014556035577,GIL LANCASTER,LOJAS AMERICANAS S/A,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,59.97,33014556,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26272,2018,43283811002608,GIL LANCASTER,KALUNGA COMERCIO E IND. GRÁFICA LTDA,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,375.35,43283811,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26273,2018,43283811004066,GIL LANCASTER,KALUNGA COMERCIO E IND.GRAFICA LTDA,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,126.60,43283811,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP
26274,2018,60437647001936,GIL LANCASTER,SILVA E BARBOSA COMERCIO DE ALIMENTOS LTDA,300585,3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,247.28,60437647,13083,GIL LANCASTER,SP,GIL LANCASTER FRAZÃO DE MORAES,250000001651,04383835830,GIL LANCASTER,DEM,ELEITO POR QP


In [74]:
despesa_selecionada = despesas_atuais_final[(despesas_atuais_final['Deputado'] == 'FERNANDO CAPEZ') & 
                              (despesas_atuais_final['Ano'] == '2016') & 
                              (despesas_atuais_final['Mes'] == '2') & 
                              (despesas_atuais_final['Tipo'] == 'J - SERVIÇOS DE COMUNICAÇÃO')]
despesa_selecionada

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
23436,2016,67641886000123,FERNANDO CAPEZ,ECT - EMP. BRAS. CORREIOS E TELEGRAFOS,300459,2,J - SERVIÇOS DE COMUNICAÇÃO,2.10,67641886,10600,FERNANDO CAPEZ,SP,FERNANDO CAPEZ,250000001510,08238377854,FERNANDO CAPEZ,PSDB,ELEITO POR QP


Verifica se algum não foi encontrado

In [75]:
nomes_naoencontrados = pd.merge(despesas_atuais, 
                        nomes_deputados_atual_legislatura, 
                        left_on='Deputado', 
                        right_on='nome_deputado',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [76]:
rdf.Deputado

82823    NaN
82824    NaN
82825    NaN
82826    NaN
82827    NaN
82828    NaN
82829    NaN
82830    NaN
82831    NaN
82832    NaN
82833    NaN
82834    NaN
82835    NaN
82836    NaN
82837    NaN
82838    NaN
82839    NaN
82840    NaN
82841    NaN
82842    NaN
82843    NaN
82844    NaN
82845    NaN
82846    NaN
82847    NaN
82848    NaN
82849    NaN
82850    NaN
Name: Deputado, dtype: object

Quantidade geral de despesas

In [78]:
despesas_atuais_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62566 entries, 0 to 62565
Data columns (total 18 columns):
Ano                     62566 non-null object
CNPJ                    62566 non-null object
Deputado                62566 non-null object
Fornecedor              62566 non-null object
Matricula               62566 non-null object
Mes                     62566 non-null object
Tipo                    62566 non-null object
Valor                   62566 non-null float64
cnpj_raiz_fornecedor    62566 non-null object
codigo_unico            62566 non-null object
nome_deputado           62566 non-null object
uf                      62566 non-null object
nome_completo           62566 non-null object
sequencial              62566 non-null object
cpf                     62566 non-null object
nome_urna               62566 non-null object
partido_eleicao         62566 non-null object
situacao                62566 non-null object
dtypes: float64(1), object(17)
memory usage: 9.1+ MB


Nomes de deputados que tiveram despesas de gabinete na atual legislatura, em quantidade

In [80]:
despesas_atuais_final['Deputado'].value_counts()

MARCIA LIA               2736
WELSON GASPARINI         2523
REINALDO ALGUZ           2223
JOOJI HATO               1907
DAVI ZAIA                1740
LUIZ CARLOS GONDIM       1693
RITA PASSOS              1633
PEDRO TOBIAS             1549
CARLOS NEDER             1539
EDSON GIRIBONI           1471
RODRIGO MORAES           1420
BARROS MUNHOZ            1390
FERNANDO CURY            1232
CORONEL TELHADA          1223
RAFAEL SILVA             1174
CHICO SARDELLI           1129
ROQUE BARBIERE           1094
CELSO NASCIMENTO         1083
FELICIANO FILHO          1020
ANALICE FERNANDES        1008
RICARDO MADALENA         1003
ROBERTO TRIPOLI           983
CEZINHA DE MADUREIRA      964
WELLINGTON MOURA          942
GIL LANCASTER             910
LUIZ TURCO                906
ITAMAR BORGES             890
CELINO CARDOSO            888
MILTON VIEIRA             883
PROFESSOR AURIEL          862
                         ... 
EDMIR CHEDID              346
LUIZ FERNANDO MACHADO     319
CARLOS GIA

# Análises

Soma das despesas

In [81]:
despesas_atuais_final['Valor'].sum()

57102331.31

In [83]:
"${:,.2f}".format(despesas_atuais_final.Valor.sum())

'$57,102,331.31'

Média

In [84]:
despesas_atuais_final['Valor'].mean()

912.6735177252822

In [85]:
"${:,.0f}".format(despesas_atuais_final.Valor.mean())

'$913'

Mediana

In [86]:
despesas_atuais_final['Valor'].median()

206.225

In [87]:
"${:,.0f}".format(despesas_atuais_final.Valor.median())

'$206'

Maiores despesas

In [88]:
despesas_atuais_final.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False).head(10)

,Deputado,Valor
47,ITAMAR BORGES,"1,225,322.51"
14,CAMPOS MACHADO,"1,155,406.50"
5,ALENCAR SANTANA BRAGA,"1,149,436.08"
35,FELICIANO FILHO,"1,134,941.68"
74,REINALDO ALGUZ,"1,132,046.81"
78,ROBERTO MASSAFERA,"1,126,564.44"
26,DAVI ZAIA,"1,113,835.46"
63,MARTA COSTA,"1,105,426.83"
85,TEONILIO BARBA,"1,099,334.21"
57,MARCIA LIA,"1,090,716.16"


Maior despesa individual

In [89]:
despesas_atuais_final[despesas_atuais_final['Valor']== despesas_atuais_final['Valor'].max()]

,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
58028,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,8,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58054,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,9,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58080,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,10,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58101,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,11,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58124,2017,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,12,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58147,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,1,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58173,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,2,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58203,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,3,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58245,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,4,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP
58273,2018,10778146820,TEONILIO BARBA,MARCELO PEREIRA NUNES,300584,5,L - LOCAÇÃO DE BENS IMÓVEIS,"9,334.61",documento_cpf_,13109,TEONILIO BARBA,SP,TEONILIO MONTEIRO DA COSTA,250000000595,02884327894,TEONILIO BARBA,PT,ELEITO POR QP


Média por ano 

In [66]:
despesas_atuais_final.groupby('Ano').mean()['Valor']

Ano
2015   900.80
2016   938.22
2017   906.06
2018   894.76
Name: Valor, dtype: float64

Quantidade de tipos de despesas

In [67]:
despesas_atuais_final['Tipo'].nunique()

13

Quantidade de despesas por Tipo

In [90]:
despesas_atuais_final['Tipo'].value_counts()

A - COMBUSTÍVEIS E LUBRIFICANTES                                      16478
I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO                    9432
H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)       7825
E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO              6937
M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS          4482
G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES E INTERNET                  3812
L - LOCAÇÃO DE BENS IMÓVEIS                                            3427
K - LOCAÇÃO DE BENS MÓVEIS                                             2685
C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE VEÍCULOS ; PEDÁGIOS     2209
D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS     2168
J - SERVIÇOS DE COMUNICAÇÃO                                            1666
F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)       1022
N - MORADIA                                                             423
Name: Tipo, 

Gastos por tipo de despesa

In [91]:
despesas_atuais_final.groupby("Tipo").Valor.sum().reset_index().sort_values("Valor", ascending=False)

,Tipo,Valor
10,L - LOCAÇÃO DE BENS IMÓVEIS,"11,727,902.34"
2,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","11,335,095.93"
0,A - COMBUSTÍVEIS E LUBRIFICANTES,"5,113,893.44"
6,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...","5,013,680.31"
4,F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTOR...,"4,264,951.74"
11,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...","3,896,369.89"
3,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,"3,815,983.28"
9,K - LOCAÇÃO DE BENS MÓVEIS,"3,390,837.53"
8,J - SERVIÇOS DE COMUNICAÇÃO,"2,991,765.99"
1,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,"2,171,163.31"


Mais gastos em quatidade em tipo D por deputado

In [92]:
despesas_atuais_final[despesas_atuais_final['Tipo']=='D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS']['Deputado'].value_counts().head(10)

WELSON GASPARINI         63
ROBERTO ENGLER           59
MILTON VIEIRA            55
GIL LANCASTER            55
ALENCAR SANTANA BRAGA    54
EDSON GIRIBONI           49
CARLOS NEDER             49
ROBERTO MORAIS           47
ANALICE FERNANDES        46
CORONEL TELHADA          42
Name: Deputado, dtype: int64

Maiores gastos em valor em tipo D por deputado

In [93]:
graficas = despesas_atuais_final[(despesas_atuais_final['Tipo'] == 'D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS')]

In [94]:
graficas.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False).head(10)

,Deputado,Valor
7,ANA DO CARMO,"327,000.00"
75,ROBERTO ENGLER,"319,533.00"
64,MILTON VIEIRA,"312,006.75"
33,ENIO TATTO,"303,420.00"
76,ROBERTO MASSAFERA,"302,800.00"
29,ED THOMAS,"289,915.00"
88,WELLINGTON MOURA,"284,964.10"
0,ABELARDO CAMARINHA,"280,415.00"
21,CEZINHA DE MADUREIRA,"267,792.80"
54,LUIZ TURCO,"263,716.92"


Graficas usadas por ANA do CARMO

In [73]:
ana_grafica = graficas[(graficas['Deputado'] == 'ANA DO CARMO')]

In [74]:
ana_grafica.reset_index().head()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
0,2987,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,1,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","8,000.00",08026465,418,ANA DO CARMO,SP,ANA MARIA DO CARMO ROSSETO,250000000510,08000475839,ANA DO CARMO,PT,ELEITO POR QP
1,2990,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,2,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","8,000.00",08026465,418,ANA DO CARMO,SP,ANA MARIA DO CARMO ROSSETO,250000000510,08000475839,ANA DO CARMO,PT,ELEITO POR QP
2,2995,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,3,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","8,000.00",08026465,418,ANA DO CARMO,SP,ANA MARIA DO CARMO ROSSETO,250000000510,08000475839,ANA DO CARMO,PT,ELEITO POR QP
3,2998,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,4,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","8,000.00",08026465,418,ANA DO CARMO,SP,ANA MARIA DO CARMO ROSSETO,250000000510,08000475839,ANA DO CARMO,PT,ELEITO POR QP
4,3004,2015,08026465000123,ANA DO CARMO,AGB PACK,300400,5,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...","8,000.00",08026465,418,ANA DO CARMO,SP,ANA MARIA DO CARMO ROSSETO,250000000510,08000475839,ANA DO CARMO,PT,ELEITO POR QP


AGB PACK COMERCIO DE EMBALAGENS E SERVICOS
VIVIANE DE CASSIA FERREIRA 
R VISCONDE DE PARNAIBA S/N - SAO PAULO - SP

ANNA GIULIA CARMO BUSCATTI
EPP
R MADRE DE DEUS 1285 APT 101 - SAO PAULO - SP

Mais gastos em quatidade em tipo L por deputado

In [77]:
despesas_atuais_final[despesas_atuais_final['Tipo']=='L - LOCAÇÃO DE BENS IMÓVEIS']['Deputado'].value_counts().head(10)

CAMPOS MACHADO           181
ROBERTO MASSAFERA        123
FERNANDO CURY            116
WELLINGTON MOURA         105
MARTA COSTA              103
ALENCAR SANTANA BRAGA     95
BARROS MUNHOZ             92
ENIO TATTO                89
GILENO GOMES              86
EDSON GIRIBONI            82
Name: Deputado, dtype: int64

Maiores gastos em valor em tipo L por deputado

In [96]:
locacao_imoveis = despesas_atuais_final[(despesas_atuais_final['Tipo'] == 'L - LOCAÇÃO DE BENS IMÓVEIS')] 

In [97]:
locacao_imoveis.groupby("Deputado").Valor.sum().reset_index().sort_values("Valor", ascending=False).head(10)

,Deputado,Valor
11,CAMPOS MACHADO,"682,636.44"
15,CELINO CARDOSO,"419,093.87"
72,WELLINGTON MOURA,"386,898.50"
2,ALDO DEMARCHI,"350,637.95"
5,ANA DO CARMO,"334,539.24"
30,FERNANDO CAPEZ,"324,797.42"
3,ALENCAR SANTANA BRAGA,"311,366.44"
31,FERNANDO CURY,"308,000.00"
22,DAVI ZAIA,"287,032.10"
69,TEONILIO BARBA,"286,617.00"


Imoveis usados por CAMPOS MACHADO

In [98]:
imoveis_campos = locacao_imoveis[(locacao_imoveis['Deputado'] == 'CAMPOS MACHADO')]

In [99]:
imoveis_campos.reset_index()

,index,Ano,CNPJ,Deputado,Fornecedor,Matricula,Mes,Tipo,Valor,cnpj_raiz_fornecedor,codigo_unico,nome_deputado,uf,nome_completo,sequencial,cpf,nome_urna,partido_eleicao,situacao
0,6394,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,"6,800.00",07813996,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
1,6395,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,"1,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
2,6396,2015,15195476845,CAMPOS MACHADO,IVETE ABBUD DIB,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,"3,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
3,6397,2015,15306494803,CAMPOS MACHADO,CLARICE ABBUD GABRIEL,300217,1,L - LOCAÇÃO DE BENS IMÓVEIS,"3,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
4,6405,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,"6,800.00",07813996,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
5,6406,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,"1,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
6,6407,2015,15195476845,CAMPOS MACHADO,IVETE ABBUD DIB,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,"3,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
7,6408,2015,15306494803,CAMPOS MACHADO,CLARICE ABBUD GABRIEL,300217,2,L - LOCAÇÃO DE BENS IMÓVEIS,"3,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
8,6414,2015,07813996000101,CAMPOS MACHADO,CMV AUTOMAÇÃO S/A,300217,3,L - LOCAÇÃO DE BENS IMÓVEIS,"6,800.00",07813996,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP
9,6415,2015,13033477844,CAMPOS MACHADO,CLAUDIA SALLUM YAZBEK,300217,3,L - LOCAÇÃO DE BENS IMÓVEIS,"1,400.00",documento_cpf_,94,CAMPOS MACHADO,SP,ANTONIO CARLOS DE CAMPOS MACHADO,250000002439,03546748891,CAMPOS MACHADO,PTB,ELEITO POR QP


In [57]:
'''
+ CMV AUTOMACAO S/A.
R CONSELHEIRO ZACARIAS, 201
VICTOR NACIM ABBUD 
GUILNAR ATALLAH ABBUD

+ CLAUDIA SALLUM YAZBEK

+ IVETE ABBUD DIB

+ CLARICE ABBUD GABRIEL

+ NORMA ABBUD SALLUM
'''

'\n+ CMV AUTOMACAO S/A.\nR CONSELHEIRO ZACARIAS, 201\nVICTOR NACIM ABBUD \nGUILNAR ATALLAH ABBUD\n\n+ CLAUDIA SALLUM YAZBEK\n\n+ IVETE ABBUD DIB\n\n+ CLARICE ABBUD GABRIEL\n\n+ NORMA ABBUD SALLUM\n'

Pega CPF de deputados e cruza com despesas - para saber se há fornecedores com CPF de deputados

In [100]:
nomes_deputados_atual_legislatura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
codigo_unico       120 non-null object
nome_deputado      120 non-null object
uf                 120 non-null object
nome_completo      120 non-null object
sequencial         120 non-null object
cpf                120 non-null object
nome_urna          120 non-null object
partido_eleicao    120 non-null object
situacao           120 non-null object
dtypes: object(9)
memory usage: 8.5+ KB


In [101]:
despesas_deputados_cpf = pd.merge(nomes_deputados_atual_legislatura, despesas_atuais_final, left_on='cpf', right_on='CNPJ')

In [104]:
despesas_deputados_cpf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 27 columns):
codigo_unico_x          7 non-null object
nome_deputado_x         7 non-null object
uf_x                    7 non-null object
nome_completo_x         7 non-null object
sequencial_x            7 non-null object
cpf_x                   7 non-null object
nome_urna_x             7 non-null object
partido_eleicao_x       7 non-null object
situacao_x              7 non-null object
Ano                     7 non-null object
CNPJ                    7 non-null object
Deputado                7 non-null object
Fornecedor              7 non-null object
Matricula               7 non-null object
Mes                     7 non-null object
Tipo                    7 non-null object
Valor                   7 non-null float64
cnpj_raiz_fornecedor    7 non-null object
codigo_unico_y          7 non-null object
nome_deputado_y         7 non-null object
uf_y                    7 non-null object
nome_completo_y 

In [115]:
despesas_deputados_cpf[['nome_urna_x','cpf_x','Fornecedor','Mes', 'Ano', 'Valor']]

,nome_urna_x,cpf_x,Fornecedor,Mes,Ano,Valor
0,ROBERTO MASSAFERA,02674980800,CIRO BERBES JUNIOR,8,2017,"2,500.00"
1,DOUTOR ULYSSES,01788949820,ULYSSES MARIO TASSINARI,10,2017,"3,656.91"
2,IGOR SOARES,28223302802,MARIA ELISA INTERNODATO NUNES VIEIRA,4,2015,"5,800.00"
3,IGOR SOARES,28223302802,MARIA ELISA INTERNODATO NUNES VIEIRA,7,2015,"5,800.00"
4,IGOR SOARES,28223302802,MARIA ELISA INTERNODATO NUNES VIEIRA,7,2015,46.45
5,IGOR SOARES,28223302802,MARIA ELISA INTERNODATO NUNES VIEIRA,1,2016,"5,800.00"
6,IGOR SOARES,28223302802,MARIA ELISA INTERNODATO NUNES VIEIRA,1,2016,46.45
